In [1]:
#LSTM
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam

# Set random seed for reproducibility
np.random.seed(42)

# Load the data
data = pd.read_csv('IBM.csv') #change the dataset here 

# Normalize the data
scaler = MinMaxScaler(feature_range=(-1, 1))
normalized_data = scaler.fit_transform(data['Close'].values.reshape(-1, 1)).flatten() # Assuming 'Close' is the column you want to predict

# Data Preprocessing Function modified for multi-step
def create_sequences(data, sequence_length, steps):
    xs, ys = [], []
    for i in range(len(data) - sequence_length - steps + 1):
        x = data[i:(i + sequence_length)]
        y = data[i + sequence_length:i + sequence_length + steps]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Create sequences with modified function
sequence_length = 1  # Updated sequence length for your use case
steps = 1  # Number of future steps to predict
X, y = create_sequences(normalized_data, sequence_length, steps)

# Split the data
split = int(len(X) * 0.9)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Reshape for LSTM [samples, time steps, features]
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Build the LSTM model
model = Sequential()
model.add(LSTM(246, input_shape=(sequence_length, 1)))
model.add(Dropout(0.2))
model.add(Dense(steps))  # The output layer should have 'steps' units if predicting multiple steps

# Compile the model
model.compile(optimizer=Adam(0.001), loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=64, verbose=1, validation_data=(X_test, y_test))

# Predicting
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Evaluate the model
mae_train = mean_absolute_error(y_train, train_predict)
rmse_train = np.sqrt(mean_squared_error(y_train, train_predict))

mae_test = mean_absolute_error(y_test, test_predict)
rmse_test = np.sqrt(mean_squared_error(y_test, test_predict))

print(f"Training Data - MAE: {mae_train}, RMSE: {rmse_train}")
print(f"Testing Data - MAE: {mae_test}, RMSE: {rmse_test}")

2024-04-04 10:20:21.675646: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/20
71/71 [==============================] - 4s 15ms/step - loss: 0.2276 - val_loss: 0.0081
Epoch 2/20
71/71 [==============================] - 0s 5ms/step - loss: 0.0918 - val_loss: 0.0018
Epoch 3/20
71/71 [==============================] - 0s 5ms/step - loss: 0.0130 - val_loss: 0.0013
Epoch 4/20
71/71 [==============================] - 0s 5ms/step - loss: 0.0032 - val_loss: 0.0013
Epoch 5/20
71/71 [==============================] - 0s 5ms/step - loss: 0.0027 - val_loss: 9.8572e-04
Epoch 6/20
71/71 [==============================] - 0s 6ms/step - loss: 0.0025 - val_loss: 8.5606e-04
Epoch 7/20
71/71 [==============================] - 0s 5ms/step - loss: 0.0024 - val_loss: 7.0191e-04
Epoch 8/20
71/71 [==============================] - 0s 5ms/step - loss: 0.0023 - val_loss: 7.1095e-04
Epoch 9/20
71/71 [==============================] - 0s 6ms/step - loss: 0.0024 - val_loss: 7.0080e-04
Epoch 10/20
71/71 [==============================] - 0s 6ms/step - loss: 0.0023 - val_loss: 6.947

In [1]:
#Adaptive SVR
import numpy as np
import pandas as pd
from sklearn.svm import SVR
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import GridSearchCV
from scipy.stats import pearsonr

# Set random seed for reproducibility
np.random.seed(42)

# Load the data
data_path = 'IBM.csv'  # Make sure to update this to your data path
data = pd.read_csv(data_path)

# Assuming 'Close' is the column you want to predict
data = data['Close'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(-1, 1))
normalized_data = scaler.fit_transform(data).flatten()

# Data Preprocessing Function modified for multi-step
def create_sequences(data, sequence_length, steps):
    xs, ys = [], []
    for i in range(len(data) - sequence_length - steps + 1):
        x = data[i:(i + sequence_length)]
        y = data[i + sequence_length:i + sequence_length + steps]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Create sequences with modified function
sequence_length = 1
steps = 1  # Number of future steps to predict
X, y = create_sequences(normalized_data, sequence_length, steps)

# Split the data
split = int(len(X) * 0.9)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Instantiate and fit the SVR model with initial parameters
svr_rbf = SVR(kernel='rbf', C=100, gamma=0.1, epsilon=.1)

# Use GridSearchCV or a similar method to adaptively find better hyperparameters based on initial performance
parameters = {'C': [1e-1, 1e0, 1e1, 1e2], 'gamma': [1e-1, 1e-2, 1e-3, 1e-4], 'epsilon': [1e-1, 1e-2, 1e-3, 1e-4]}
grid_search = GridSearchCV(svr_rbf, parameters, cv=5)
grid_search.fit(X_train, y_train.ravel())

best_svr = grid_search.best_estimator_

# Predictions with the optimized model
train_predict = best_svr.predict(X_train)
test_predict = best_svr.predict(X_test)

# Evaluate the model using MAE and RMSE on the scaled data
mae_train = mean_absolute_error(y_train, train_predict)
rmse_train = np.sqrt(mean_squared_error(y_train, train_predict))

mae_test = mean_absolute_error(y_test, test_predict)
rmse_test = np.sqrt(mean_squared_error(y_test, test_predict))

print(f"Training Data - MAE: {mae_train}, RMSE: {rmse_train}")
print(f"Testing Data - MAE: {mae_test}, RMSE: {rmse_test}")

2024-04-04 10:22:42.242898: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/20
71/71 [==============================] - 4s 14ms/step - loss: 0.0508 - val_loss: 0.0049
Epoch 2/20
71/71 [==============================] - 0s 7ms/step - loss: 0.0046 - val_loss: 0.0032
Epoch 3/20
71/71 [==============================] - 0s 5ms/step - loss: 0.0041 - val_loss: 0.0030
Epoch 4/20
71/71 [==============================] - 1s 7ms/step - loss: 0.0037 - val_loss: 0.0028
Epoch 5/20
71/71 [==============================] - 0s 5ms/step - loss: 0.0035 - val_loss: 0.0028
Epoch 6/20
71/71 [==============================] - 1s 8ms/step - loss: 0.0034 - val_loss: 0.0025
Epoch 7/20
71/71 [==============================] - 0s 5ms/step - loss: 0.0031 - val_loss: 0.0023
Epoch 8/20
71/71 [==============================] - 0s 6ms/step - loss: 0.0031 - val_loss: 0.0022
Epoch 9/20
71/71 [==============================] - 1s 9ms/step - loss: 0.0030 - val_loss: 0.0022
Epoch 10/20
71/71 [==============================] - 0s 5ms/step - loss: 0.0028 - val_loss: 0.0020
Epoch 11/20
71/71 

In [1]:
#CNN-LSTM
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error, mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Conv1D, MaxPooling1D, Flatten, TimeDistributed, Reshape
from tensorflow.keras.optimizers import Adam

# Set random seed for reproducibility
np.random.seed(42)

# Load the data from a CSV file
data_path = 'IBM.csv'  # Update this to the correct path where IBM.csv is located
data = pd.read_csv(data_path)

# Assuming 'Close' is the column you want to predict, adjust if needed
data = data['Close'].values.reshape(-1, 1)

# Normalize the data
scaler = MinMaxScaler(feature_range=(-1, 1))
normalized_data = scaler.fit_transform(data).flatten()

# Data Preprocessing Function for multi-step
def create_sequences(data, sequence_length, steps):
    xs, ys = [], []
    for i in range(len(data) - sequence_length - steps + 1):
        x = data[i:(i + sequence_length)]
        y = data[i + sequence_length:i + sequence_length + steps]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys)

# Create sequences with modified function
sequence_length = 7  # This is the window size of your sequence
steps = 1  # This is the prediction horizon
X, y = create_sequences(normalized_data, sequence_length, steps)

# Split the data
split = int(len(X) * 0.9)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

# Reshape for CNN-LSTM [samples, subsequences, timesteps, features]
# For this example, we're keeping the model simple without changing the shape too much
# Normally, you would reshape your data to have multiple subsequences for the CNN to process
subsequences = 1
timesteps = X_train.shape[1] // subsequences
X_train = X_train.reshape((X_train.shape[0], subsequences, timesteps, 1))
X_test = X_test.reshape((X_test.shape[0], subsequences, timesteps, 1))

# Define the CNN-LSTM model
model = Sequential()

# CNN layers within TimeDistributed wrapper to process each subsequence
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=3, activation='relu'), input_shape=(None, timesteps, 1)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))

# LSTM layers
model.add(LSTM(124, return_sequences=False))
model.add(Dropout(0.2))

# Final dense layers
model.add(Dense(50, activation='relu'))
model.add(Dense(steps))  # Adjust 'steps' if you're predicting more than one future step

# Compile the model
model.compile(optimizer=Adam(0.001), loss='mean_squared_error')

# Train the model
model.fit(X_train, y_train, epochs=20, batch_size=64, verbose=1, validation_data=(X_test, y_test))

# Predictions
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

# Evaluate the model using MAE and RMSE
mae_train = mean_absolute_error(y_train, train_predict)
rmse_train = np.sqrt(mean_squared_error(y_train, train_predict))

mae_test = mean_absolute_error(y_test, test_predict)
rmse_test = np.sqrt(mean_squared_error(y_test, test_predict))

print(f"Training Data - MAE: {mae_train}, RMSE: {rmse_train}")
print(f"Testing Data - MAE: {mae_test}, RMSE: {rmse_test}")

2024-04-04 10:22:42.242898: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/20
71/71 [==============================] - 4s 14ms/step - loss: 0.0508 - val_loss: 0.0049
Epoch 2/20
71/71 [==============================] - 0s 7ms/step - loss: 0.0046 - val_loss: 0.0032
Epoch 3/20
71/71 [==============================] - 0s 5ms/step - loss: 0.0041 - val_loss: 0.0030
Epoch 4/20
71/71 [==============================] - 1s 7ms/step - loss: 0.0037 - val_loss: 0.0028
Epoch 5/20
71/71 [==============================] - 0s 5ms/step - loss: 0.0035 - val_loss: 0.0028
Epoch 6/20
71/71 [==============================] - 1s 8ms/step - loss: 0.0034 - val_loss: 0.0025
Epoch 7/20
71/71 [==============================] - 0s 5ms/step - loss: 0.0031 - val_loss: 0.0023
Epoch 8/20
71/71 [==============================] - 0s 6ms/step - loss: 0.0031 - val_loss: 0.0022
Epoch 9/20
71/71 [==============================] - 1s 9ms/step - loss: 0.0030 - val_loss: 0.0022
Epoch 10/20
71/71 [==============================] - 0s 5ms/step - loss: 0.0028 - val_loss: 0.0020
Epoch 11/20
71/71 

In [4]:
#CNN-BiLSTM-ECA
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Dense, LSTM, Bidirectional, Conv1D, MaxPooling1D, Input,
    GlobalAveragePooling1D, Reshape, Permute, TimeDistributed,
    Flatten, RepeatVector, concatenate, Add
)
from tensorflow.keras.optimizers import Adam
from math import sqrt
import tensorflow as tf

# Define the Attention mechanism
def attention_block(inputs):
    # inputs.shape = (batch_size, time_steps, input_dim)
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim, -1))(a) # this line is not useful. It's already 2D tensor
    a = Dense(time_steps, activation='softmax')(a)
    if len(inputs.shape) == 3:
        a = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = Multiply()([inputs, a])
    return output_attention_mul

# Load and preprocess data
data = pd.read_csv('IBM.csv')
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)
data.sort_index(inplace=True)

# Normalization
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(data.values)
data_normalized = pd.DataFrame(scaled_data, index=data.index, columns=data.columns)

# Generate time-series data
time_steps = 5
X, Y = [], []
for i in range(time_steps, len(data_normalized)):
    X.append(data_normalized.iloc[i - time_steps:i].values)
    Y.append(data_normalized.iloc[i]['Close'])
X, Y = np.array(X), np.array(Y)

# Train-test split
train_size = int(len(X) * 0.9)
X_train, X_test = X[:train_size], X[train_size:]
Y_train, Y_test = Y[:train_size], Y[train_size:]

# Model configuration
input_layer = Input(shape=(time_steps, X.shape[2]))
cnn_blocks = []
for _ in range(3): # Three CNN layers
    cnn_out = Conv1D(filters=64, kernel_size=1, activation='relu')(input_layer)
    cnn_out = MaxPooling1D(pool_size=2)(cnn_out)
    cnn_blocks.append(cnn_out)

# Merge CNN outputs
merged_cnn = Add()(cnn_blocks)

# BiLSTM layers after CNN
bi_lstm = Bidirectional(LSTM(50, return_sequences=True))(merged_cnn)
bi_lstm = Bidirectional(LSTM(50, return_sequences=True))(bi_lstm)

# Attention layer after BiLSTM
attention_mul = attention_block(bi_lstm)
attention_flatten = Flatten()(attention_mul)

# Final Dense layers and output
dense_out = Dense(50, activation='relu')(attention_flatten)
output_layer = Dense(1)(dense_out)

# Compile model
model = Model(inputs=input_layer, outputs=output_layer)
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Train model
history = model.fit(X_train, Y_train, epochs=20, batch_size=32, validation_split=0.2, verbose=1)

# Predictions
predictions = model.predict(X_test)
predictions = scaler.inverse_transform(predictions) # Only if Y was scaled

# Calculate errors
mse = mean_squared_error(Y_test, predictions)
mae = mean_absolute_error(Y_test, predictions)
rmse = sqrt(mse)
print(f'MSE: {mse}, MAE: {mae}, RMSE: {rmse}')

Epoch 1/20
126/126 [==============================] - 5s 10ms/step - loss: 0.0320 - val_loss: 0.0060
Epoch 2/20
126/126 [==============================] - 1s 6ms/step - loss: 6.9254e-04 - val_loss: 0.0027
Epoch 3/20
126/126 [==============================] - 1s 5ms/step - loss: 5.5953e-04 - val_loss: 0.0014
Epoch 4/20
126/126 [==============================] - 1s 7ms/step - loss: 4.8722e-04 - val_loss: 8.3745e-04
Epoch 5/20
126/126 [==============================] - 1s 6ms/step - loss: 4.5453e-04 - val_loss: 7.3234e-04
Epoch 6/20
126/126 [==============================] - 1s 5ms/step - loss: 4.3784e-04 - val_loss: 0.0014
Epoch 7/20
126/126 [==============================] - 1s 5ms/step - loss: 4.1729e-04 - val_loss: 7.7616e-04
Epoch 8/20
126/126 [==============================] - 1s 5ms/step - loss: 4.2618e-04 - val_loss: 5.7864e-04
Epoch 9/20
126/126 [==============================] - 1s 5ms/step - loss: 4.1134e-04 - val_loss: 5.9449e-04
Epoch 10/20
126/126 [==========================